In [1]:
import json 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
fname = '/home/azureuser/localfiles/code/slm-finetuning/fc-finetune-hf/src/data/fc_commands.json'

with open(fname) as f:
    data = json.load(f)

In [3]:
data['set_temperature']['incomplete_commands'][0]

[{'incomplete_command': 'Set the temperature.',
  'modified_incorrect_function_call': 'set_temperature()',
  'is_correct': True,
  'reason': 'The INCOMPLETE_USER_COMMAND does not mention any parameters or values, and the MODIFIED_INCORRECT_FUNCTION_CALL correctly does not include any parameters or values either.'},
 {'incomplete_command': 'Change the temperature',
  'modified_incorrect_function_call': 'set_temperature()',
  'is_correct': True,
  'reason': "The INCOMPLETE_USER_COMMAND mentions only the 'temperature' parameter without specifying any value. The MODIFIED_INCORRECT_FUNCTION_CALL does not include any parameters or values, which is consistent with the INCOMPLETE_USER_COMMAND."},
 {'incomplete_command': 'Please adjust the temperature.',
  'modified_incorrect_function_call': 'set_temperature()',
  'is_correct': True,
  'reason': "The parameter 'temperature' is mentioned in the INCOMPLETE_USER_COMMAND, but its value is not mentioned. Therefore, it is correct that the parameter i

In [4]:
def get_fn_df(fn_name):
    fn_data = data[fn_name]
    fn_df = pd.DataFrame()
    for d in fn_data['incomplete_commands']:
        df = pd.DataFrame(d)
        fn_df = pd.concat([fn_df, df])
    return fn_df

set_temp_df = get_fn_df('set_temperature')
set_temp_df
        

,incomplete_command,modified_incorrect_function_call,is_correct,reason
0,Set the temperature.,set_temperature(),True,The INCOMPLETE_USER_COMMAND does not mention a...
1,Change the temperature,set_temperature(),True,The INCOMPLETE_USER_COMMAND mentions only the ...
2,Please adjust the temperature.,set_temperature(),True,The parameter 'temperature' is mentioned in th...
3,Make it 33 degrees inside the car,set_temperature(temperature=33),True,The parameter 'temperature' with value '33' is...
4,Turn the heat up.,set_temperature(),False,The parameter 'temperature' is mentioned in th...
0,Set the temperature.,set_temperature(),True,The INCOMPLETE_USER_COMMAND does not mention a...
1,Can you make it ?,set_temperature(),False,The MODIFIED_INCORRECT_FUNCTION_CALL does not ...
2,"Adjust the temperature, please.",set_temperature(),True,The INCOMPLETE_USER_COMMAND does not mention a...
3,Bring the inside temperature up.,set_temperature(),False,The parameter 'temperature' is mentioned in th...
4,Make the car's temperature.,set_temperature(),True,The INCOMPLETE_USER_COMMAND does not provide a...


In [5]:
fn_names = list(data.keys())
df_main = pd.DataFrame()
for fn_name in fn_names:
    df = get_fn_df(fn_name)
    df['fn_name'] = fn_name
    df_main = pd.concat([df_main, df])
    df_main = df_main.reset_index(drop=True)

df_main = df_main[df_main['incomplete_command'] != '']

In [6]:
# get "is_correct" sum grouped by fn_name
df_correct_sum = df_main.groupby('fn_name')['is_correct'].sum().reset_index().sort_values('is_correct', ascending=False).rename(columns={'is_correct': 'correct_count'})
df_count = df_main.groupby('fn_name').size().reset_index().rename(columns={0: 'count'})
df_summary = df_correct_sum.merge(df_count, on='fn_name')
df_summary['accuracy'] = df_summary['correct_count'] / df_summary['count']

df_summary


,fn_name,correct_count,count,accuracy
0,set_cruise_control,9,10,0.9
1,control_window,8,10,0.8
2,set_temperature,7,10,0.7
3,start_engine,5,5,1.0
4,set_fan_speed,5,10,0.5
5,play_music,4,5,0.8
6,toggle_headlights,4,10,0.4
7,adjust_seat,3,10,0.3
8,adjust_wiper_speed,3,10,0.3
9,control_ambient_lighting,3,10,0.3


In [7]:
df_main[df_main['is_correct'] == False].to_dict(orient='records')

[{'incomplete_command': 'Turn the heat up.',
  'modified_incorrect_function_call': 'set_temperature()',
  'is_correct': False,
  'reason': "The parameter 'temperature' is mentioned in the INCOMPLETE_USER_COMMAND but is not present in the MODIFIED_INCORRECT_FUNCTION_CALL. Additionally, the MODIFIED_INCORRECT_FUNCTION_CALL does not mention any parameters or values that can be inferred from the INCOMPLETE_USER_COMMAND.",
  'fn_name': 'set_temperature'},
 {'incomplete_command': 'Can you make it ?',
  'modified_incorrect_function_call': 'set_temperature()',
  'is_correct': False,
  'reason': "The MODIFIED_INCORRECT_FUNCTION_CALL does not mention any parameters or values from the INCOMPLETE_USER_COMMAND. Additionally, it includes a parameter 'temperature' which is not present in the INCOMPLETE_USER_COMMAND.",
  'fn_name': 'set_temperature'},
 {'incomplete_command': 'Bring the inside temperature up.',
  'modified_incorrect_function_call': 'set_temperature()',
  'is_correct': False,
  'reason'